In [ ]:
import os
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

import os
import datetime
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%matplotlib inline

In [ ]:
def get_ds(index,start_ts=0,step=120):
    index_tmp=index*step+start_ts
    date=datetime.datetime.fromtimestamp(index_tmp).strftime('%Y-%m-%d %H:%M:%S')
    date=pd.Timestamp(date)
    return date

def norm(value):
    # if value>0.05:
    #     return 0.05
    if value<0.005:
        return 0.0108-0.0023
    else:
        return value

df=pd.read_csv('datasets/data.csv')

df['index']=range(df.__len__())
df['date']=df['index'].map(get_ds)

# df['ts_norm']=df['index'].map(get_norm_time)

# del df['index']
del df['id']

df['e_mu_current']=df['e_mu_current'].map(norm)

# df['OT']=np.log(df['OT'])
# df['e_nu_1']=np.log(df['e_nu_1'])
# df['e_nu_2']=np.log(df['e_nu_2'])
# df['q_mu']=np.log(df['q_mu'])
# df['q_nu1']=np.log(df['q_nu1'])
# df['q_nu2']=np.log(df['q_nu2'])
df['e_mu_current']=np.log(df['e_mu_current'])


# df['e_mu_current']=(df['e_mu_current']-np.mean(df['e_mu_current']))/np.std(df['e_mu_current'])

plt.plot(df['e_mu_current'])
plt.show()

In [ ]:
print(np.max(np.exp(df['e_mu_current'])))
print(np.min(np.exp(df['e_mu_current'])))
print(np.mean((df['e_mu_current'])))
print(np.std(df['e_mu_current']))

In [ ]:
outliers=pd.read_csv('./../datasets_qrate/outliers_info.csv')[['index','class','steps_to_anomaly']]

def replace_str(value):
    if value=='anomaly':
        return 1
    else:
        return 0

outliers['class']=outliers['class'].map(replace_str)
outliers

In [ ]:
df_outliers=df.iloc[outliers['index']]

df_non_outliers = pd.merge(df,df_outliers, on='index',how='left')
df_non_outliers = df[~df.isin(df_outliers)].dropna()
df_non_outliers

In [ ]:
del df_non_outliers['index']
del df_outliers['index']

print(len(df_non_outliers))
print(len(df_non_outliers)*0.1)
print(len(df_outliers))
print(len(df_outliers)*0.1)

In [ ]:
df_non_outliers.iloc[:-18160].to_csv('datasets/non_outliers_step=120_log_inf_train=0.9.csv',index=False,)
df_non_outliers.iloc[-18160:].to_csv('datasets/non_outliers_step=120_log_inf_test=0.1.csv',index=False,)

df_outliers.iloc[:-300].to_csv('datasets/outliers_step=120_log_inf_train=0.9.csv',index=False,)
df_outliers.iloc[-300:].to_csv('datasets/outliers_step=120_log_inf_test=0.1.csv',index=False,)

In [ ]:
df_outliers

In [ ]:
# df=pd.read_csv('datasets/outliers_step=120_log_inf_train=0.9.csv')
plt.plot(df['e_mu_current'])
plt.show()


In [ ]:
df

In [ ]:

# fix_seed = 2021
# np.random.seed(fix_seed)
# torch.manual_seed(fix_seed)
# np.random.seed(fix_seed)

In [ ]:

args = dotdict()
args.target = 'e_mu_current'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.gpu = 0
args.lradj = 'type1'
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

args.freq = 't'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.is_trainging = True
args.root_path = './datasets/'
# args.data_path ='non_outliers_step=120_log_inf_train=0.9.csv' 
args.data_path ='outliers_step=120_log_inf_train=0.9.csv' 
args.model_id='qber_96_48_120_loginf_t_outliers'
args.model = 'Autoformer'
args.data = 'custom'
args.features = 'MS'
args.seq_len = 96
args.label_len = 48
args.pred_len = 1
args.e_layers = 6
args.d_layers = 6
args.n_heads = 8
args.factor = 3
args.enc_in = 6
args.dec_in =6
args.c_out = 6
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 1
args.distil = False
args.output_attention = True
args.patience= 3
args.learning_rate = 0.0001
args.batch_size = 64
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'
args.train_epochs = 10

exp = Exp_Main(args)

In [ ]:

setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 0)
print(setting)

In [ ]:
exp.train(setting)
# exp.test(setting)

In [ ]:
exp.args.root_path = './datasets/'
exp.args.data_path = 'data_step=60_test=0.1.csv'

prediction=exp.predict(setting, True)

In [ ]:
torch.cuda.empty_cache()


In [ ]:
# exp_name='qber_96_48_120_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'
# exp_name='qber_96_48_120_loginf_t_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'
exp_name='qber_96_48_120_loginf_s_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'
exp_name='test_qber_96_48_120_loginf_s_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'

data_pred = np.load(f'./results/{exp_name}/pred.npy')

data_true = np.load(f'./results/{exp_name}/true.npy')


In [ ]:
start=0
end=5000
plt.figure(figsize=(20,5))
plt.plot(data_pred[start:end,0,0])
plt.plot(data_true[start:end,0,0])
plt.legend(['predicted','true'])
plt.show()

In [ ]:
start=8200
end=8700
plt.figure(figsize=(20,5))
plt.plot(data_pred[start:end,0,0])
plt.plot(data_true[start:end,0,0])
plt.legend(['predicted','true'])
plt.show()